In [9]:
import pandas as pd
import numpy as np


## FONCTIONS DE PREPROCESS GENERALES

In [11]:
import pandas as pd
import numpy as np
import re


def transform_date(df, columns: str):
    df[columns] = pd.to_datetime(df[columns])
    # Extraire l'année, le mois et le jour de la semaine
    df['Années'] = df[columns].dt.year
    df['Mois'] = df[columns].dt.month
    df['Heure'] = df[columns].dt.hour
    df['Jour de la semaine'] = df[columns].dt.day_name()
    # Mapper les jours de la semaine en français
    jours_semaine_fr = {
        'Monday': 'LUNDI',
        'Tuesday': 'MARDI',
        'Wednesday': 'MERCREDI',
        'Thursday': 'JEUDI',
        'Friday': 'VENDREDI',
        'Saturday': 'SAMEDI',
        'Sunday': 'DIMANCHE'
    }
    # Mapper les mois en français
    mois_fr = {
        1: 'JANVIER',
        2: 'FEVRIER',
        3: 'MARS',
        4: 'AVRIL',
        5: 'MAI',
        6: 'JUIN',
        7: 'JUILLET',
        8: 'AOUT',
        9: 'SEPTEMBRE',
        10: 'OCTOBRE',
        11: 'NOVEMBRE',
        12: 'DECEMBRE'
    }
    # Remplacer les numéros de mois par leur équivalent en français
    df['Mois'] = df['Mois'].map(mois_fr)
    # Remplacer les noms des jours par leur équivalent en français
    df['Jour de la semaine'] = df['Jour de la semaine'].map(jours_semaine_fr)
    return df

def clean_cell_number(df, columns:str):
    df[columns] = df[columns].astype(str)
    df[columns] = df[columns].str.split(',').str[0]
    df[columns] = df[columns].replace(r'^0693', '262693', regex=True)
    df[columns] = df[columns].replace(r'^0692', '262692', regex=True)
    df[columns] = df[columns].replace(r'^06', '336', regex=True)
    df[columns] = df[columns].replace(r'^07', '337', regex=True)
    df[columns] = df[columns].replace(r'^02', '2622', regex=True)
    return df

def reset_accent(chaine):
    accent_fr = {
    'é': 'e', 'è': 'e', 'ê': 'e', 'ë': 'e',
    'à': 'a', 'â': 'a', 'ä': 'a',
    'ç': 'c',
    'î': 'i', 'ï': 'i',
    'ô': 'o', 'ö': 'o',
    'ù': 'u', 'û': 'u', 'ü': 'u',
    'ÿ': 'y',
    # Ajout des majuscules
    'É': 'E', 'È': 'E', 'Ê': 'E', 'Ë': 'E',
    'À': 'A', 'Â': 'A', 'Ä': 'A',
    'Ç': 'C',
    'Î': 'I', 'Ï': 'I',
    'Ô': 'O', 'Ö': 'O',
    'Ù': 'U', 'Û': 'U', 'Ü': 'U'
}
    for accent, sans_accent in accent_fr.items():
        chaine = chaine.replace(accent, sans_accent)
    return chaine


def clean_city(df, columns: str):
    df[columns] = df[columns].astype(str)
    df = df.rename(columns={columns: 'VILLE'})
    df['VILLE']= df['VILLE'].str.upper()
    df['VILLE']= df['VILLE'].str.replace("-", " ")
    df['VILLE']= df['VILLE'].str.replace("SAINT", "ST")
    df['VILLE']= df['VILLE'].str.replace("SAINTE", "STE")
    df['VILLE']= df['VILLE'].str.replace("L'", "")
    df['VILLE'] = df['VILLE'].apply(reset_accent)
    df['VILLE'] = df['VILLE'].str.replace(r'\s+', ' ', regex=True).str.strip()
    return df


def convert_date(date_str):
    # Extraire le fuseau horaire avec une expression régulière
    match = re.search(r'UTC([+-]\d)', date_str)
    if match:
        utc_offset = int(match.group(1))  # Récupérer le décalage horaire
    else:
        utc_offset = 0  # Valeur par défaut si aucun fuseau horaire n'est trouvé
    # Retirer le fuseau horaire et convertir en datetime
    date_without_tz = date_str.split(' UTC')[0]  # Retirer ' UTC+X'
    dt = pd.to_datetime(date_without_tz, format='%d/%m/%Y - %H:%M:%S')
    # Ajuster le fuseau horaire à UTC+4
    dt = dt + pd.Timedelta(hours=(4 - utc_offset))  # Ajustement basé sur l'offset
    return dt

# Fonction pour nettoyer les chaînes
def clean_number(number_str):
    # Utiliser une expression régulière pour supprimer le préfixe et le suffixe
    return re.sub(r'^\=\("\s*|\s*"\)$', '', number_str)

def replace_unknown_ville(row):
    reunion_postal_codes = {
    "97400": "ST DENIS",
    "97410": "ST PIERRE",
    "97411": "BOIS DE NEFLES",
    "97412": "BRAS PANON",
    "97413": "CILAOS",
    "97414": "ENTRE DEUX",
    "97416": "LA CHALOUPE",
    "97417": "ST BERNARD",
    "97418": "LA PLAINE DES CAFRES",
    "97419": "LA POSSESSION",
    "97420": "LE PORT",
    "97421": "LA RIVIERE ST LOUIS",
    "97422": "LA SALINE",
    "97423": "LE GUILLAUME",
    "97424": "PITON ST LEU",
    "97425": "LES AVRIONS",
    "97426": "LES TROIS BASSINS",
    "97427": "L'ETANG SALE",
    "97428": "LA NOUVELLE",
    "97429": "PETITE ILE",
    "97430": "LE TAMPON",
    "97431": "LA PLAINE DES PALMISTES",
    "97432": "LA RAVINE DES CABRIS",
    "97433": "SALAZIE - HELL BOURG",
    "97434": "ST GILLES LES BAINS",
    "97435": "BERNICA",
    "97436": "ST LEU",
    "97437": "STE ANNE",
    "97438": "STE MARIE",
    "97439": "STE ROSE",
    "97440": "ST ANDRE",
    "97441": "STE SUZANNE",
    "97442": "BASSE VALLEE",
    "97450": "ST LOUIS",
    "97460": "ST PAUL",
    "97470": "ST BENOIT"
}
    if row['VILLE'] == 'ville inconnue':
        return reunion_postal_codes.get(row['CODE POSTAL'], 'ville inconnue')  # Remplace par la ville correspondante ou garde 'ville inconnue'
    return row['VILLE']


def extract_city(address):
    if isinstance(address, str):  # Vérifie si l'adresse est une chaîne
        # Regex pour capturer la ville après le code postal (5 chiffres)
        match = re.search(r'\d{5}\s+(.*)', address)
        if match:
            return match.group(1).strip()  # Retourne la ville sans espaces superflus
    return None  # Retourne None si aucune correspondance n'est trouvée ou si l'adresse n'est pas une chaîne



## FONCTIONS TRAITEMENT DES DONNEES SRR

In [ ]:
def preprocess_data(file1, file2, sheet_name=0):
    expected_columns_file1 = ["Type d'appel", "Abonné", "Correspondant", "Date", "Durée", "CIREF", "IMEI", "IMSI"]
    expected_columns_file2 = ["CIREF", "Adresse", "Comp. adresse", "Code postal", "Bureau Distributeur", "Coordonnée X", "Coordonnée Y"]
    df1 = pd.read_excel(file1, sheet_name=sheet_name, dtype={"Abonné": str, "Correspondant": str, "IMEI": str, "IMSI": str, "CIREF": str, "Durée": str, "Type d'appel" : str})
    df2 = pd.read_excel(file2, dtype= {"CIREF": str, "Adresse" : str, "Comp. adresse" : str, "Code postal" : str, "Bureau Distributeur" : str, "Coordonnée X": str, "Coordonnée Y": str})
    available_columns_1 = df1.columns.tolist()
    # Filtrer les colonnes attendues qui sont disponibles
    filtered_columns_1 = list(set(expected_columns_file1) & set(available_columns_1))
    available_columns_2 = df2.columns.tolist()
    # Filtrer les colonnes attendues qui sont disponibles
    filtered_columns_2 = list(set(expected_columns_file2) & set(available_columns_2))
    df1 = df1[filtered_columns_1]
    df1['Abonné'] = df1['Abonné'].ffill()
    df1['Abonné'] = df1['Abonné'].bfill()
    df2 = df2[filtered_columns_2]
    df = df1.merge(df2, on="CIREF", how="left")
    deleted_columns =['Critère Recherché_x', 'Commentaire_x', '3ème interlocuteur', 'Nature Correspondant',
       'Nature 3ème interlocuteur', 'GCI_x', 'EGCI_x', 'NGCI_x', 'Code PLMN',
       'Volume de données montant', 'Volume de données descendant',"Opérateur d'itinérance", 'Indicateur RO', 'Décalage horaire',
       'Service de Base', 'IPV4 VO Wifi', 'IPV6 VO Wifi',
       'Port Source VO Wifi', 'Critère Recherché_y', 'Commentaire_y', 'GCI_y',
       'EGCI_y', 'NGCI_y', 'Système', 'Nom du site', 'Code zone', 'Coordonnée Z', 'Début asso. CIREF/GCI', 'Fin asso. CIREF/GCI']
    for i in df.columns.tolist():
        if i in deleted_columns:
            df.drop(i, axis=1, inplace=True)
    if 'Date' in df.columns:
        df = transform_date(df, columns='Date')
    if 'Abonné' in df.columns:
        df = clean_cell_number(df, 'Abonné')
    if 'Correspondant' in df.columns:
        df = clean_cell_number(df, 'Correspondant')
    if "Type d'appel" in df.columns:
        df["Type d'appel"] = df["Type d'appel"].astype(str)
        df["Type d'appel"] = df["Type d'appel"].str.upper()
        df["Type d'appel"] = df["Type d'appel"].apply(reset_accent)
    if 'Bureau Distributeur' in df.columns:
        df = clean_city(df, columns='Bureau Distributeur')
    if 'Adresse' in df.columns and 'Code postal' in df.columns:
        df['adresse_complete'] = df['Adresse'] + " " + df['Code postal'] + " " + df['VILLE']
        df['adresse_complete'] = df['adresse_complete'].astype(str)
        df['adresse_complete']= df['adresse_complete'].str.upper()
        df['adresse_complete']= df['adresse_complete'].str.replace(r'\s+', ' ', regex=True)
        df['adresse_complete'] = df['adresse_complete'].apply(reset_accent)
        df = df.drop(columns=['Adresse', 'Comp. adresse', 'Code postal'])
        df= df.rename(columns={'adresse_complete': 'Adresse'})
    df = df.fillna("INDETERMINE")
    definitive_columns = ["Type d'appel", "Abonné", "Correspondant", "Date", "Durée", "CIREF", "IMEI", "IMSI", "Adresse", "Ville", 'Années', 'Mois', 'Heure', 'Jour de la semaine', "Coordonnée X", "Coordonnée Y"]
    no_accent_columns = [reset_accent(i) for i in definitive_columns]
    final_columns = [i.upper() for i in no_accent_columns]
    for old_columns, new_columns in zip(definitive_columns, final_columns):
        df = df.rename(columns={old_columns : new_columns})
    return df

In [25]:
file1 = "/home/antoine/telephony_insight_project/streamlit/app/source/models_files/SRR_Detcom_Imei_20241219004095_D2.xls"
file2 = "/home/antoine/telephony_insight_project/streamlit/app/source/models_files/SRR_Ident_Cellule_20241219004098_D2.xls"

In [26]:
result = preprocess_data(file1, file2)

In [20]:
result.head()

,CORRESPONDANT,ABONNE,IMSI,TYPE D'APPEL,DATE,DUREE,CIREF,IMEI,COORDONNEE X,COORDONNEE Y,VILLE,ANNEES,MOIS,HEURE,JOUR DE LA SEMAINE,ADRESSE
0,.sl2sfr,262693947742,647108506505762,SESSION GPRS UMTS,2024-01-01 07:56:57,359,400004626,35589425639400,135760,60320,ST PAUL,2024.0,JANVIER,7.0,LUNDI,"31, CHEMIN DES FLEURS LE GUILLAUME 97423 ST PAUL"
1,123,262693947742,647108506505762,CONSULTATION DU REPONDEUR,2024-01-01 08:02:58,5,970007289,35589425639400,132901,58751,ST PAUL,2024.0,JANVIER,8.0,LUNDI,IMMEUBLE EUCALYPTUS 84 CHEMIN DEPARTEMENTALE 6...
2,123,262693947742,647108506505762,CONSULTATION DU REPONDEUR,2024-01-01 08:02:58,5,970007289,35589425639400,132901,58751,ST PAUL,2024.0,JANVIER,8.0,LUNDI,IMMEUBLE EUCALYPTUS 84 CHEMIN DEPARTEMENTALE 6...
3,123,262693947742,647108506505762,CONSULTATION DU REPONDEUR,2024-01-01 08:03:11,1,970017275,35589425639400,134280,58580,ST PAUL,2024.0,JANVIER,8.0,LUNDI,102 ROUTE DE FATIMA LE BERNICA SAINT GILLES LE...
4,123,262693947742,647108506505762,CONSULTATION DU REPONDEUR,2024-01-01 08:03:11,1,970017275,35589425639400,134280,58580,ST PAUL,2024.0,JANVIER,8.0,LUNDI,102 ROUTE DE FATIMA LE BERNICA SAINT GILLES LE...
